In [45]:
import numpy as np
import time
import sys
import os
from joblib import Parallel, delayed
import multiprocessing

In [46]:
min(num_cores,nlayer)

26

In [ ]:
base_name = 'depth0'
folder = '/srv/home/chenyu/antisite/PRISMATIC/Depth0_TiltSeries/0.6mrad/'
cbed_list_unique = [img for img in os.listdir(folder) if (base_name in img) and ('FP1.mrc' in img) and ('_X1_Y1_' in img)]
nlayer = len(cbed_list_unique)
num_cores = multiprocessing.cpu_count()
print(num_cores)
Parallel(n_jobs=min(num_cores,nlayer))(delayed(averageFP)(folder,base_name,ilayer) for ilayer in range(nlayer))

40


In [42]:
def averageFP(folder, base_name, ilayer):
    layer_name = '_slice'+ str(ilayer+1)
    # only work for CBEDs in slice ilayer+1
    cbed_list_unique = [img for img in os.listdir(folder) if (base_name in img) and ('FP1.mrc' in img) 
                        and (layer_name in img)]
    cbed_list_all = [img for img in os.listdir(folder) if ((base_name in img) and ('FP' in img)) and ('.mrc' in img) 
                     and (layer_name in img)]
    shape = readCBEDfromMRC(folder + cbed_list_unique[0]).shape
    
    for pos in cbed_list_unique:
        cmp = pos.rsplit('_',1)
        cbed_pos = cmp[0][:]
        tmp_array = np.zeros(readCBEDfromMRC(folder + pos).shape,dtype=np.float32)
        fp = 0
        
        for cbed in cbed_list_all:
            if (cbed_pos+'_') in cbed:
                tmp_array += readCBEDfromMRC(folder+cbed)
#                 os.remove(cbed)
                fp += 1
        
        tmp_array /= fp
        np.save(folder+cbed_pos+'_FPavg',tmp_array)

In [2]:
def readCBEDfromMRC(fname):
    f = open(fname, 'rb')
    count = 10
    b = f.read(count*4)
    data_types = {0:np.uint8,1:np.intc,2:np.float32,6:np.uint16}
    c = np.frombuffer(b, dtype=np.int32,count=count)
    data_size = c[:3]
    data_type = data_types[c[3]]

    f.seek(1024)

    count = data_size[0]*data_size[1]*data_size[2]
    cbed = f.read(count*4)
    cbed_arr = np.frombuffer(cbed,dtype=data_type,count=count)
    cbed_arr = np.reshape(cbed_arr,tuple(data_size))
    f.close
    f.closed
    return cbed_arr

In [20]:
cmp = pos.rsplit('_',1)
cbed_pos = cmp[0][:]

In [21]:
cbed_pos

'depth0_slice8_X67_Y35'

In [25]:
img

NameError: name 'img' is not defined